In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from matplotlib import patches

from PIL import Image
import pylab as pyl

import tensorflow as tf
import keras
import sklearn
import h5py
import shutil
import scipy
import cython
import mido
import pytest
import six
import librosa

import IPython.display

from pathlib import Path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


In [2]:
pip install --upgrade librosa

Requirement already up-to-date: librosa in /usr/local/lib/python3.6/dist-packages (0.7.0)


In [3]:
!pip install madmom

In [4]:
import madmom
import librosa
import librosa.display

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')
!pwd
import os.path
from os import path

/content/drive/My Drive/Colab Notebooks


In [0]:
def trouver_la_liste(beat_frames,annot_frames):
  
  i=0
  liste=[]
  
  #trouver le premier annot sup ou egal au premier downbeat et apres ok pour faire tous les cas!
  
  for k in range (len(annot_frames)): 
    
    annot=annot_frames[k]
    
    while i<=len(beat_frames)-1 and beat_frames[i]<=annot:
      i=i+1
      
    if (i-1)<=len(beat_frames)-1  and  beat_frames[i-1]<=annot:
      liste.append(i-1)
      i=i+1
    else:
      i=i+1
 
  return(liste)

In [0]:
y,sr=librosa.load('/content/drive/My Drive/Colab Notebooks/Jazzomat Database/Weimar Jazz Database - original tracks/MilesDavis_BluesByFive_Orig.wav')

In [0]:
pwd

'/content/drive/My Drive/Colab Notebooks'

In [0]:
os.mkdir('pourlepowerpoint')

In [13]:
cd /content/drive/My Drive/Colab Notebooks/pourlepowerpoint

/content/drive/My Drive/Colab Notebooks/pourlepowerpoint


In [0]:
os.mkdir('SSMs_2')

In [0]:
pwd

'/content/drive/My Drive/Colab Notebooks/pourlepowerpoint'

In [0]:
  i=0
#for i in idSALAMITest:
  val=i
  F=100
    
  y,sr=librosa.load('/content/drive/My Drive/Colab Notebooks/Jazzomat Database/Weimar Jazz Database - original tracks/JohnColtrane_Countdown_Orig.wav')
  y_later_use=y
  proc = madmom.features.downbeats.DBNDownBeatTrackingProcessor(beats_per_bar=[3, 4], fps=100)
  act = madmom.features.downbeats.RNNDownBeatProcessor()(y)
  estimated = proc(act)
  beats = estimated[:,0]
  beats_frames=librosa.core.time_to_frames(beats, sr=sr*2)
  
   
  # Set the hop length; at 22050 Hz, 512 samples ~= 23ms
  hop_length = 512

  # Separate harmonics and percussives into two waveforms
  y_harmonic, y_percussive = librosa.effects.hpss(y)


  # Compute MFCC features from the raw signal
  mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
  beats_frames=np.insert(beats_frames,0, 0)
  beats_frames=np.append(beats_frames,len(mfcc.T)-1)
  
  annot_sec1=[23, 113]
  annot_frames1=librosa.core.time_to_frames(annot_sec1, sr=sr)
  liste1=trouver_la_liste(beats_frames,annot_frames1)
    

  # And the first-order differences (delta features)
  mfcc_delta = librosa.feature.delta(mfcc)

  # Stack and synchronize between beat events
  # This time, we'll use the mean value (default) instead of median
  beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beats_frames)

  # Compute chroma features from the harmonic signal
  chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

  # Aggregate chroma features between beat events
  # We'll use the median value of each feature between beat frames
  beat_chroma = librosa.util.sync(chromagram,
                                beats_frames,
                                aggregate=np.median)

  R_aff_chroma = librosa.segment.recurrence_matrix(beat_chroma, mode='affinity',metric='cosine',self=True)
  R_aff_mfcc = librosa.segment.recurrence_matrix(beat_mfcc_delta, mode='affinity',metric='cosine',self=True)
  
  
  R_aff_chroma=np.pad(R_aff_chroma, (50,50), 'constant', constant_values=(0, 0))
  R_aff_mfcc=np.pad(R_aff_mfcc, (50,50), 'constant', constant_values=(0, 0))
  
  listefusion=liste1
  listefusion=sorted(listefusion)    
      
  
  dico_sousmatrice_SMJ1={}
  for k in range (int(F/2),int(len(R_aff_chroma)-F/2)):
    m= np.zeros((F,F))
    for x in range (0,F-1):
      for y in range (0,F-1):
        m[x][y]=R_aff_chroma[int(x+(k-F/2))][int(y+(k-F/2))]
              
    dico_sousmatrice_SMJ1[k-int(F/2)]=m
    
    
  dico_sousmatrice_SMJ2={}
  for k in range (int(F/2),int(len(R_aff_chroma)-F/2)):
    m= np.zeros((F,F))
    for x in range (0,F-1):
      for y in range (0,F-1):
        m[x][y]=R_aff_mfcc[int(x+(k-F/2))][int(y+(k-F/2))]
              
    dico_sousmatrice_SMJ2[k-int(F/2)]=m 
            
    
    
  #partie où on sauve avec chromas&mfcss
  cl=np.zeros((100,100))
  cl=Image.fromarray(np.uint8(cl*255), 'L')
  for j in range(len(dico_sousmatrice_SMJ1)):
    im1=dico_sousmatrice_SMJ1[j]
    im2=dico_sousmatrice_SMJ2[j]
    im_chroma=Image.fromarray(np.uint8(im1*255), 'L')
    im_mfcc=Image.fromarray(np.uint8(im2*255), 'L')
    merged=Image.merge("RGB",(cl, im_mfcc, im_chroma))
    
    if j in listefusion:
      merged.save('/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/SSMs_2/SSM_segment_sousmatrice'+str(val)+'_'+str(j)+'.jpeg')
    else:
      merged.save('/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/SSMs_2/SSM_sousmatrice'+str(val)+'_'+str(j)+'.jpeg') 
      
      
 
   

In [0]:
import os
import os.path
from os import path
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from matplotlib import patches
import keras

from PIL import Image
import pylab as pyl

from keras import models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

import time
import tensorflow as tf

In [0]:
base_dir = '/content/drive/My Drive/Colab Notebooks/Salami preprocessing_beatsync/SSMs'
train_dir = '/content/drive/My Drive/Colab Notebooks/Salami preprocessing_beatsync/SSMs/Train/'
validation_dir = '/content/drive/My Drive/Colab Notebooks/Salami preprocessing_beatsync/SSMs/Validation/'
test_dir= '/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/Test2/'


# Directory with our training SMJ_nonsolo pictures
train_SMJ_nonsolo_dir = '/content/drive/My Drive/Colab Notebooks/Salami preprocessing_beatsync/SSMs/Train/Non_segment'

# Directory with our training SMJ_solo pictures
train_SMJ_solo_dir = '/content/drive/My Drive/Colab Notebooks/Salami preprocessing_beatsync/SSMs/Train/Segment'


# Directory with our test SMJ_nonsolo pictures
validation_SMJ_nonsolo_dir = os.path.join(validation_dir, 'Non_segment')

# Directory with our test SMJ_solo pictures
validation_SMJ_solo_dir = os.path.join(validation_dir, 'Segment')

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        directory=train_dir,  # This is the source directory for training images
        target_size=(100,100),  # All images will be resized to 100*100
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        directory=validation_dir,
        target_size=(100, 100),
        batch_size=128,
        class_mode='binary')


test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(100, 100),
    batch_size=1,
    class_mode=None)


Found 138280 images belonging to 2 classes.
Found 13024 images belonging to 2 classes.
Found 201 images belonging to 1 classes.


In [19]:
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/SSMbeatsync-weights-improvement-07-0.88.h5')


W0903 15:54:17.020405 140591444678528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()

In [21]:
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

201/201 [==============================] - 1s 7ms/step


In [0]:
y_pred=[1 if y>=0.5 else 0 for y in pred]

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred]

In [0]:
import pickle
with open('/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/test2_predictionsSSMwithdownbeatsync', 'wb') as f:
    pickle.dump(predictions, f)

In [0]:
predictions=pickle.load(open('/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/test2_predictionsSSMwithdownbeatsync', 'rb'))

In [26]:
predictions

['Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'Non_segment',
 'No

In [0]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/test2_predictionsSSMwithdownbeatsync.csv",index=False)

In [28]:
!pip install --upgrade -q gspread 
!pip install gspread-dataframe 

In [0]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [0]:
from google.colab import auth 
auth.authenticate_user()
from oauth2client.client import GoogleCredentials 
gc=gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
sheet1=gc.open_by_url('https://docs.google.com/spreadsheets/d/1IfLWgDvLE7vorpwXHyHLWd-rDVe5ZWKJB1tO2Z0f5JQ/edit#gid=1167787435')

In [0]:
ws=sheet1.worksheet('test2_predictionsSSMwithdownbeatsync')

In [33]:
file=get_as_dataframe(ws)
file

,Filename,Predictions
0,Test2/SSM_segment_sousmatrice0_164.jpeg,Non_segment
1,Test2/SSM_segment_sousmatrice0_35.jpeg,Non_segment
2,Test2/SSM_sousmatrice0_0.jpeg,Non_segment
3,Test2/SSM_sousmatrice0_1.jpeg,Non_segment
4,Test2/SSM_sousmatrice0_10.jpeg,Non_segment
5,Test2/SSM_sousmatrice0_100.jpeg,Non_segment
6,Test2/SSM_sousmatrice0_101.jpeg,Non_segment
7,Test2/SSM_sousmatrice0_102.jpeg,Non_segment
8,Test2/SSM_sousmatrice0_103.jpeg,Segment
9,Test2/SSM_sousmatrice0_104.jpeg,Non_segment


In [0]:
segment=[]

for i in range(len(file)):
  if file["Predictions"][i]=="Segment":
    segment.append(file["Filename"][i])

In [35]:
segment

['Test2/SSM_sousmatrice0_103.jpeg',
 'Test2/SSM_sousmatrice0_123.jpeg',
 'Test2/SSM_sousmatrice0_134.jpeg',
 'Test2/SSM_sousmatrice0_178.jpeg',
 'Test2/SSM_sousmatrice0_182.jpeg',
 'Test2/SSM_sousmatrice0_191.jpeg',
 'Test2/SSM_sousmatrice0_34.jpeg',
 'Test2/SSM_sousmatrice0_45.jpeg',
 'Test2/SSM_sousmatrice0_78.jpeg',
 'Test2/SSM_sousmatrice0_8.jpeg',
 'Test2/SSM_sousmatrice0_9.jpeg',
 'Test2/SSM_sousmatrice0_93.jpeg',
 'Test2/SSM_sousmatrice0_95.jpeg']

In [0]:
segments=pd.DataFrame({"Filename":segment})
segments.to_csv("/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/test2_segments-predictionsSSMwithdownbeatsync.csv",index=False)

In [0]:
  import csv
  
  j=0
  
  val=j
  sheet2=gc.open_by_url('https://docs.google.com/spreadsheets/d/1Qz2S30B_OztsZhPlRnYKlKZaUf283AfRBkHrozxEm8o/edit#gid=1666346134')
  ws=sheet2.worksheet('test2_segments-predictionsSSMwithdownbeatsync')
  file=get_as_dataframe(ws)
  
  segment=[]
  for i in range(len(file)):
      
    if "Test2/SSM_sousmatrice"+str(j)+"_" in file["Filename"][i]:
      t="Test2/SSM_sousmatrice"+str(j)+"_"
      t2=".jpeg"
      txt1 = (file["Filename"][i]).replace(t,' ')
      txt1 = txt1.replace(t2,' ')
      segment.append(int(txt1))
    if "Test2/SSM_segment_sousmatrice"+str(j)+"_" in file["Filename"][i]:
      t3="Test2/SSM_segment_sousmatrice"+str(j)+"_"
      t2=".jpeg"
      txt2 = (file["Filename"][i]).replace(t3,' ')
      txt2 = txt2.replace(t2,' ')
      segment.append(int(txt2))
    
    
  segments=pd.DataFrame({"Filename":segment})
  segments.to_csv("/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/segments2/"+str(j)+".csv",index=False)
  fichier="/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/segments2/"+str(j)+".csv"
  file=open(fichier,'r')
  reader=csv.reader(file)
  
  l=list(reader)
  l=l[1:len(l)]
  
  
  liste=[]
  for i in range (len(l)):
  
    liste.append(int(l[i][0]))
  
  
  y,sr=librosa.load('/content/drive/My Drive/Colab Notebooks/Jazzomat Database/Weimar Jazz Database - original tracks/JohnColtrane_Countdown_Orig.wav')
  proc = madmom.features.downbeats.DBNDownBeatTrackingProcessor(beats_per_bar=[3, 4], fps=100)
  act = madmom.features.downbeats.RNNDownBeatProcessor()(y)
  estimated = proc(act)
  beats = estimated[:,0]
  beats_frames=librosa.core.time_to_frames(beats, sr=sr*2)


  chromagram = librosa.feature.chroma_cqt(y=y,
                                        sr=sr)
  
  beats_frames=np.insert(beats_frames,0, 0)
  beats_frames=np.append(beats_frames,len(chromagram.T)-1)

  liste=sorted(liste) 
  
  beatsfusion=[]
  for i in range(len(liste)):
      beatsfusion.append(beats_frames[liste[i]])


  liste_times=librosa.core.frames_to_time(beatsfusion, sr=sr)
  
  
  if len(liste_times)>1:
    col1=liste_times[0:len(liste_times)-1]
    col2=liste_times[1:len(liste_times)]
    
    fichier='/content/drive/My Drive/Colab Notebooks/pourlepowerpoint/times2/'+str(val)+'.csv'
    file=open(fichier,'w')
    writer=csv.writer(file,delimiter=',')
    fin=len(col1)
    for i in range(fin):
      writer.writerow((col1[i],col2[i]))
    
    file.close() 
    
    